In [1]:
import os
from openai import OpenAI
# Credentials is a file created to store all my credentials
from credentials import *

# Creating an object of Credentials class

In [2]:
cred = Credentials()
all_cred = cred.get_credential()

# Creating OpenAI client object to access the API

In [3]:
client = OpenAI(
    # This is the default and can be omitted
    api_key=all_cred["RAG_OPENAI_KEY"]
) 

In [4]:
# Function that would take prompt and return the response for a particular role
# Here it is taking only a single role i.e. user here
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message.content

In [5]:
get_completion(prompt="What is the day today?", model="gpt-3.5-turbo")

'I am an AI assistant and I do not have the ability to know the current date. Please check your calendar or device for the current date.'

<b style="color:red; font-size:1.5em"> Here for every prompt going into the message we are getting only one response. But for the chatbot, we need to make it recursive so that multiple roles can be passed recursively as can be seen in  <b style="color:green"> point-6 </b> inside <b style="color:green">ProgrammingEngineering.md</b> file</b>

# Creating a function so that multiple messages from different roles can be passed

In [6]:
# The function is kind of similar to the one defined before. It's just instead of a single message we are passing multiple here
# In the above function we were passing prompt as the parameter, but here we are passing messages as the parameter since we have to 
# pass content related to multiple roles
def get_completion_from_messages(messages, model="gpt-3.5-turbo"):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message

In [7]:
# Multiple messages are as follows:
messages =  [  
{'role':'system', 'content':'You are an assistant that speaks like Shakespeare.'},    
{'role':'user', 'content':'tell me a joke'},   
{'role':'assistant', 'content':'Why did the chicken cross the road'},   
{'role':'user', 'content':'I don\'t know'}  
]

In [8]:
response = get_completion_from_messages(messages)

In [9]:
response

ChatCompletionMessage(content='To get to the other side, good sir!', refusal=None, role='assistant', function_call=None, tool_calls=None)

 As we can see from above cell, the output has come from <b>assistant</b>

In [10]:
response.role, response.content

('assistant', 'To get to the other side, good sir!')

# Let's see if model remembers our information

In [11]:
messages =  [  
{'role':'system', 'content':'You are a friendly chatbot and will interact with users from different backgrounds.'},    
{'role':'user', 'content':'Hi, my name is Vaibhav'}  ]

response = get_completion_from_messages(messages)
print(response.role, "\n" ,response.content)

assistant 
 Hello Vaibhav! It's nice to meet you. How are you doing today?


In [12]:
messages =  [  
{'role':'system', 'content':'You are a friendly chatbot and will interact with users from different backgrounds.'},    
{'role':'user', 'content':'Yes,  can you remind me, What is my name?'}  ]

response = get_completion_from_messages(messages)
print(response.role, "\n" ,response.content)

assistant 
 I'm sorry, but I don't have access to personal information about users. You can tell me your name if you'd like!


<b style="color:red; font-size:1.5em"> From the above two cells, it can be seen that model doesn't remember our past conversation, So, we have to deal with it!!   </b>

<b style="color:green; font-size:1.5em"> SOLUTION: So, if we want our model to remember the earlier conversation, we need to pass it as an input to the model as can be seen below.</b>

In [13]:
messages =  [  
{'role':'system', 'content':'You are a friendly chatbot and will interact with users from different backgrounds.'},
{'role':'user', 'content':'Hi, my name is Vaibhav'},
{'role':'assistant', 'content': "Hi Isa! It's nice to meet you. \
Is there anything I can help you with today?"},
{'role':'user', 'content':'Yes, you can remind me, What is my name?'}  ]

response = get_completion_from_messages(messages)
print(response.role, "\n" ,response.content)

assistant 
 Your name is Vaibhav.


<b> So, as can be seen from the above cell when passing the earlier conversation, the model is easily able to provide the response.  </b>

# Let's build the chatbot having UI based for the user such that all past conversation will keep on appending in the list so that we don't manually have to append

In [14]:
def collect_messages(inp):
    # User giving query
    prompt = inp
    
    # Appending the user's query in the context list which stores all the past conversation
    context.append({'role':'user', 'content':f"{prompt}"})
    # Getting response of the query asked by the user
    response = get_completion_from_messages(context).content 
    # Appending the Agent's response in the context list which stores all the past conversation
    context.append({'role':'assistant', 'content':f"{response}"})

    # Printing the response for the user's query
    print(response)


In [15]:
# Telling what the system is, so that it tells the agent to reply accordingly
context = [ {'role':'system', 'content':'You are a friendly chatbot and will interact with users from different backgrounds.'} ]  # accumulate messages

inp = ''

# Looping until user enters to exit
while inp != "exit":
    inp = input("Enter the query user!!")
    collect_messages(inp)
    

Enter the query user!! Hi! This is Vaibhav, and I am here to know about the popular places in London?


Hello Vaibhav! I'd be happy to help you discover popular places in London. London is a vibrant city with a rich history and plenty of attractions to explore. Some popular places to visit in London include:

1. The British Museum: Home to a vast collection of art and artifacts from around the world.
2. The Tower of London: A historic castle on the River Thames with a fascinating past.
3. Buckingham Palace: The official residence of the British monarch and a must-see for visitors.
4. The London Eye: A giant observation wheel offering stunning views of the city skyline.
5. Covent Garden: A lively area with shops, restaurants, and street performers.
6. The West End: London's famous theater district, known for its world-class shows.
7. Hyde Park: A beautiful green space in the heart of the city, perfect for a relaxing stroll.
8. The Shard: An iconic skyscraper with an observation deck offering panoramic views of London.

These are just a few of the many amazing places to explore in London. 

Enter the query user!! Which one should I go first?


If you're looking to start your London adventure with a must-see attraction, I would recommend visiting Buckingham Palace first. It's an iconic symbol of the British monarchy and a popular tourist destination. You can witness the Changing of the Guard ceremony, explore the beautiful gardens, and take in the grandeur of the palace from the outside.

After visiting Buckingham Palace, you can explore nearby attractions such as St. James's Park, The Mall, and Westminster Abbey. This area is rich in history and offers a great introduction to London's royal heritage.

Of course, the choice of where to go first ultimately depends on your interests and preferences. If you have a particular attraction in mind that you're excited about, feel free to prioritize that one. Enjoy your time exploring London!


Enter the query user!! I am with my friend Louis and Ryan


...

As an AI chatbot, I don't have the ability to know personal information about users unless it has been shared with me during our conversation. I am here to provide information, assistance, and engage in friendly conversation with you. If there's anything specific you'd like to know or talk about, feel free to share, and I'll do my best to help!


Enter the query user!! Do you know my name?


Yes, you mentioned earlier that your name is Vaibhav. How can I assist you further, Vaibhav?


Enter the query user!! exit


If you have any more questions in the future, feel free to come back. Have a great day, Vaibhav! Goodbye!
